# Sono più importanti per la classifica elements o components?

## 1. Importo dataset

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px

In [13]:
data = pd.read_csv("final.csv")
print(data.shape) #214531, 19 rows, cols
data.head(1)

(214491, 19)


,competition,program,performance_id,athlete_name,athlete_nation,rank,total_segment_score,total_element_score,total_component_score,total_deductions,aspect_id,aspect_desc,element_order,element_base_value,section,aspect_final_score,judge_name,judge_nation,judge_score
0,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,Evgenia MEDVEDEVA,RUS,1,78.92,42.0,36.92,0.0,00034b9414,Transitions,NaN,NaN,components,9.07,Agita ABELE,LAT,9.0


Colonne importanti:
* rank 
* total_segment_score	
* total_element_score	
* total_component_score

Righe importanti per performance_id $\implies$ elimino doppioni

In [14]:
# Elimino le righe con stessa performance_id:
data = data.drop_duplicates(subset=['performance_id']).reset_index()

# Elimino le colonne che non mi servono più
data.drop('section', axis=1, inplace=True)
data.drop('total_deductions', axis=1, inplace=True)
data.drop('aspect_id', axis=1, inplace=True)
data.drop('aspect_desc', axis=1, inplace=True)
data.drop('element_order', axis=1, inplace=True)
data.drop('element_base_value', axis=1, inplace=True)
data.drop('aspect_final_score', axis=1, inplace=True)
data.drop('judge_name', axis=1, inplace=True)
data.drop('athlete_name', axis=1, inplace=True)
data.drop('athlete_nation', axis=1, inplace=True)
data.drop('judge_nation', axis=1, inplace=True)
data.drop('judge_score', axis=1, inplace=True)

data.head(1)

,index,competition,program,performance_id,rank,total_segment_score,total_element_score,total_component_score
0,0,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,1,78.92,42.0,36.92


Creo le nuove 4 categorie:

In [15]:
# quali sono le categorie di program?
data['program'].value_counts()

MEN SHORT PROGRAM               246
LADIES SHORT PROGRAM            244
LADIES FREE SKATING             218
MEN FREE SKATING                218
ICE DANCE SHORT DANCE           207
ICE DANCE FREE DANCE            185
PAIRS SHORT PROGRAM             164
PAIRS FREE SKATING              150
JUNIOR MEN SHORT PROGRAM         12
JUNIOR ICE DANCE FREE DANCE      12
JUNIOR PAIRS SHORT PROGRAM       12
JUNIOR ICE DANCE SHORT DANCE     12
JUNIOR PAIRS FREE SKATING        12
JUNIOR MEN FREE SKATING          12
JUNIOR LADIES FREE SKATING       11
JUNIOR LADIES SHORT PROGRAM      11
Name: program, dtype: int64

In [16]:
# creo una colonna caregory in cui sono presenti le 8 categorie: ladies, men, pairs, ice_dance + free
# create a list of our conditions
conditions = [
    
    (data['program'] == "LADIES SHORT PROGRAM") |
    (data['program'] == "JUNIOR LADIES SHORT PROGRAM"),

    (data['program'] == "MEN SHORT PROGRAM") |
    (data['program'] == "JUNIOR MEN SHORT PROGRAM"),

    (data['program'] == "PAIRS SHORT PROGRAM") |
    (data['program'] == "JUNIOR PAIRS SHORT PROGRAM"),

    (data['program'] == "ICE DANCE SHORT DANCE") |
    (data['program'] == "JUNIOR ICE DANCE SHORT DANCE") |
    (data['program'] == "ICE DANCE FREE DANCE") |
    (data['program'] == "JUNIOR ICE DANCE FREE DANCE"),

    (data['program'] == "JUNIOR LADIES FREE SKATING") |
    (data['program'] == "LADIES FREE SKATING") ,

    (data['program'] == "MEN FREE SKATING") |
    (data['program'] == "JUNIOR MEN FREE SKATING"),

    (data['program'] == "PAIRS FREE SKATING") |
    (data['program'] == "JUNIOR PAIRS FREE SKATING")

    ]

# create a list of the values we want to assign for each condition
values = ['Ladies','Men','Pairs','Ice dance','Ladies free','Men free','Pairs free']

# create a new column and use np.select to assign values to it using our lists as arguments
data['category'] = np.select(conditions, values)

# display updated DataFrame
data.head()

,index,competition,program,performance_id,rank,total_segment_score,total_element_score,total_component_score,category
0,0,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,1,78.92,42.00,36.92,Ladies
1,12,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,4764e4873e,24,45.97,24.77,21.20,Ladies
2,24,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,877e5c34bb,2,74.39,39.90,34.49,Ladies
3,36,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,53868b204a,17,51.47,29.04,22.43,Ladies
4,48,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,da4b091d04,3,72.40,36.80,35.60,Ladies


In [17]:
# Calcola quante categorie presenta la colonna 'rank' in 'data' per ogni 'competition' e 'program':
ranking = data.groupby(['competition','program'])['rank'].nunique() #Count number of distinct elements in specified axis
ranking.head(20)

competition                                     program                     
Grand Prix Final 2017 Senior and Junior         ICE DANCE FREE DANCE             6
                                                ICE DANCE SHORT DANCE            6
                                                JUNIOR ICE DANCE FREE DANCE      6
                                                JUNIOR ICE DANCE SHORT DANCE     6
                                                JUNIOR LADIES FREE SKATING       6
                                                JUNIOR LADIES SHORT PROGRAM      6
                                                JUNIOR MEN FREE SKATING          6
                                                JUNIOR MEN SHORT PROGRAM         6
                                                JUNIOR PAIRS FREE SKATING        6
                                                JUNIOR PAIRS SHORT PROGRAM       6
                                                LADIES FREE SKATING              6
          

## 2. Bubble plot - valori assoluti

* colore è la categoria
* plottare una baseline y=x
* migliorare graficamente
* pensare summary
* cambiare size

In [18]:
# bubble1_data = px.scatter(
#     data, y="total_component_score", x="total_element_score",
#     color="program", size="rank", 
#     title='Total component score vs total element score',
#     log_x=False, size_max=36
#     )
# bubble1_data.show()

In [19]:
# Ordino data secondo il rank in modo rescente:
data = data.sort_values(by=['rank'], ascending=True)
# Create a new column with the inverse of the rank:
data['rank_inv'] = 1/data['rank']
data.head(1)

,index,competition,program,performance_id,rank,total_segment_score,total_element_score,total_component_score,category,rank_inv
0,0,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,1,78.92,42.0,36.92,Ladies,1.0


In [20]:
bubble_data = px.scatter(
    data, y="total_component_score", x="total_element_score",
    color="category", size="rank_inv", 
    title='Rank: components vs elements score for each performance',
    log_x=False, size_max=60,
    labels={
        "category": "Category",  
        "total_component_score": "Components score", 
        "total_element_score": "Elements score",
        "rank": "Rank", 
        "program": "Program",
    },
    hover_name="performance_id", hover_data={"total_component_score":True, "total_element_score":True, "rank":True, "program":True, "rank_inv":False},
    category_orders={ # replaces default order by column name
        "category": ['Ladies','Ladies free','Men','Men free','Pairs','Pairs free','Ice dance']
    },
    template="simple_white",
    width=1000,
    height=600
    )
bubble_data.update_traces(
    marker=dict(symbol="star", line=dict(width=0.5, color='DarkSlateGrey')),
    selector=dict(mode="markers"),
    
)
# Aggiungo y = x
x = [0,10,20,30,40,50,60,70,80,90,100,110,120]
bubble_data.add_scatter(x=x, y=x, mode="lines", line_color="black", line_width=2, opacity=1, line_dash="dot", name="equilibrium")
bubble_data.show()

## 3. Bubble plot - valori relativi

In [21]:
# Aggiungi colonna con la differenza tra component_score e element_score per ogni performance_id in percentuale rispetto al totale total_segment_score:
data['weight_elem'] = data['total_element_score']/data['total_segment_score']
data['weight_comp'] = data['total_component_score']/data['total_segment_score']
data.head()

,index,competition,program,performance_id,rank,total_segment_score,total_element_score,total_component_score,category,rank_inv,weight_elem,weight_comp
0,0,ISU European Figure Skating Championships 2017,LADIES SHORT PROGRAM,b639d77459,1,78.92,42.00,36.92,Ladies,1.0,0.532184,0.467816
837,103731,Grand Prix Final 2017 Senior and Junior,LADIES FREE SKATING,5e8023f5f0,1,147.03,76.61,70.42,Ladies free,1.0,0.521050,0.478950
805,100830,ISU World Figure Skating Championships 2017,ICE DANCE SHORT DANCE,8e7983756f,1,82.43,43.74,38.69,Ice dance,1.0,0.530632,0.469368
793,99910,ISU GP Rostelecom Cup 2017,ICE DANCE SHORT DANCE,43f5771db8,1,77.30,40.23,37.07,Ice dance,1.0,0.520440,0.479560
782,98280,ISU GP Audi Cup of China 2016,MEN FREE SKATING,58a25fecce,1,196.31,104.31,93.00,Men free,1.0,0.531353,0.473741


In [23]:
bubble_data2 = px.scatter(
    data, y="weight_comp", x="weight_elem",
    color="category", size="rank_inv", 
    title='Rank: components vs elements score for each performance',
    log_x=False, size_max=60,
    labels={
        "weight_comp": "Components weight",
        "weight_elem": "Elements weight",
        "category": "Category",  
        "total_component_score": "Components score", 
        "total_element_score": "Elements score",
        "rank": "Rank", 
        "program": "Program",
    },
    hover_name="performance_id", hover_data={"total_component_score":True, "total_element_score":True, "rank":True, "program":True, "rank_inv":False},
    category_orders={ # replaces default order by column name
        "category": ['Ladies','Ladies free','Men','Men free','Pairs','Pairs free','Ice dance']
    },
    template="simple_white",
    width=1000,
    height=600
    )
bubble_data2.update_traces(
    marker=dict(symbol="star", line=dict(width=0.5, color='DarkSlateGrey')),
    selector=dict(mode="markers"),
    
)
bubble_data2.layout.xaxis.tickformat = '0%'
bubble_data2.layout.yaxis.tickformat = '0%'
# Aggiungo y = x
bubble_data2.add_vline(x=0.5, line_width=2, line_dash="dash", line_color="black", opacity=1)
bubble_data2.add_hline(y=0.5, line_width=2, line_dash="dash", line_color="black", opacity=1)
bubble_data2.show()

Con questi dati è evidente che non c'è differenza tra Ladies e Ladies Free nel trend, quindi raggruppo i dati in una sola categoria.

## 4. Bubble plot - new version

In [24]:
bubble_data3 = px.scatter(
    data, x="weight_elem", y="rank",
    color="category", #size="rank_inv", 
    title='Rank: components vs elements score for each performance',
    log_x=False,
    labels={
        "category": "Category",  
        "total_segment_score": "Total score",
        "total_component_score": "Components score", 
        "total_element_score": "Elements score",
        "rank": "Rank", 
        "program": "Program",
        "weight_elem": "Weight of elements"
    },
    hover_name="performance_id", hover_data={"total_component_score":True, "total_element_score":True, "rank":True, "program":True, "rank_inv":False, "total_segment_score":True},
    category_orders={ # replaces default order by column name
        "category": ['Ladies','Ladies free','Men','Men free','Pairs','Pairs free','Ice dance']
    },
    template="simple_white",
    width=1000,
    height=600
    )
bubble_data3.update_traces(
    marker=dict(symbol="circle", line=dict(width=0.5, color='DarkSlateGrey'), size=10),
    selector=dict(mode="markers"),
    
)
bubble_data3.layout.xaxis.tickformat = '0%'
# #Aggiungo y = x
bubble_data3.add_vline(x=0.5, line_width=2, line_dash="dash", line_color="black", opacity=1, name="equilibrium")
# x = [0,10,20,30,40,50,60,70,80,90,100,110,120]
# bubble_data2.add_scatter(x=x, y=x, mode="lines", line_color="black", line_width=2, opacity=1, line_dash="dot", name="equilibrium")
bubble_data3.show()

## 5. Bubble plot - summerized

In [25]:
medie = data.groupby(['category', 'rank'])['weight_elem', 'weight_comp'].mean().reset_index()
# Ordino data secondo il rank in modo crescente:
medie = medie.sort_values(by=['rank'], ascending=True)
# Create a new column with the inverse of the rank:
medie['rank_inv'] = 1/medie['rank']
medie.colnames = ['category', 'rank', 'weight_elem', 'weight_comp', 'rank_inv']
medie.head()

<ipython-input-25-3eb5b75b227c>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-25-3eb5b75b227c>:6: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



,category,rank,weight_elem,weight_comp,rank_inv
0,Ice dance,1,0.510014,0.489986,1.0
32,Ladies,1,0.532529,0.467471,1.0
93,Men,1,0.554580,0.447137,1.0
129,Men free,1,0.528687,0.474170,1.0
153,Pairs,1,0.545739,0.454937,1.0


In [26]:
medie.tail()

,category,rank,weight_elem,weight_comp,rank_inv
66,Ladies,35,0.559813,0.461473,0.028571
127,Men,35,0.494702,0.514048,0.028571
67,Ladies,36,0.536779,0.484667,0.027778
128,Men,36,0.465464,0.560196,0.027778
68,Ladies,37,0.513816,0.610369,0.027027


In [28]:
bubble_medie = px.scatter(
    medie, y="weight_comp", x="weight_elem",
    color="category", size="rank_inv", 
    title='Rank: components vs elements score for each performance',
    size_max=40,
    labels={
        "weight_comp": "Components weight mean",
        "weight_elem": "Elements weight mean",
        "rank": "Rank", 
        "category": "Category"
    },
    hover_data={"rank":True, "rank_inv":False},
    category_orders={ # replaces default order by column name
        "category": ['Ladies','Ladies free','Men','Men free','Pairs','Pairs free','Ice dance']
    },
    template="simple_white",
    width=1000,
    height=600
    )
bubble_medie.update_traces(
    marker=dict(symbol="star", line=dict(width=0.5, color='DarkSlateGrey')),
    selector=dict(mode="markers"),
    
)
bubble_medie.layout.xaxis.tickformat = '0%'
bubble_medie.layout.yaxis.tickformat = '0%'
# Aggiungo y = x
bubble_medie.add_vline(x=0.5, line_width=2, line_dash="dash", line_color="black", opacity=1)
bubble_medie.add_hline(y=0.5, line_width=2, line_dash="dash", line_color="black", opacity=1)
bubble_medie.show()

## 6. ~~Bubble plot - 4 categories~~

In [ ]:
# elimina la colonna category
# data.drop(columns=['category'], inplace=True)

# conditions = [
#     (data['program'] == "LADIES SHORT PROGRAM") |
#     (data['program'] == "JUNIOR LADIES SHORT PROGRAM") |
#     (data['program'] == "JUNIOR LADIES FREE SKATING") |
#     (data['program'] == "LADIES FREE SKATING"),

#     (data['program'] == "MEN SHORT PROGRAM") |
#     (data['program'] == "JUNIOR MEN SHORT PROGRAM") |
#     (data['program'] == "MEN FREE SKATING") |
#     (data['program'] == "JUNIOR MEN FREE SKATING"),

#     (data['program'] == "PAIRS SHORT PROGRAM") |
#     (data['program'] == "JUNIOR PAIRS SHORT PROGRAM") |
#     (data['program'] == "PAIRS FREE SKATING") |
#     (data['program'] == "JUNIOR PAIRS FREE SKATING"),

#     (data['program'] == "ICE DANCE SHORT DANCE") |
#     (data['program'] == "JUNIOR ICE DANCE SHORT DANCE") |
#     (data['program'] == "ICE DANCE FREE DANCE") |
#     (data['program'] == "JUNIOR ICE DANCE FREE DANCE"),

#     ]

# # create a list of the values we want to assign for each condition
# values = ['Ladies','Men','Pairs','Ice dance']

# # create a new column and use np.select to assign values to it using our lists as arguments
# data['category'] = np.select(conditions, values)

# # display updated DataFrame
# data.head()

In [ ]:
# bubble_data2_1 = px.scatter(
#     data, y="weight_comp", x="weight_elem",
#     color="category", size="rank_inv", 
#     title='Rank: components vs elements score for each performance',
#     log_x=False, size_max=60,
#     labels={
#         "weight_comp": "Components weight",
#         "weight_elem": "Elements weight",
#         "category": "Category",  
#         "total_component_score": "Components score", 
#         "total_element_score": "Elements score",
#         "rank": "Rank", 
#         "program": "Program",
#     },
#     hover_name="performance_id", hover_data={"total_component_score":True, "total_element_score":True, "rank":True, "program":True, "rank_inv":False},
#     category_orders={ # replaces default order by column name
#         "category": ['Ladies','Ladies free','Men','Men free','Pairs','Pairs free','Ice dance']
#     },
#     template="simple_white",
#     width=1000,
#     height=600
#     )
# bubble_data2_1.update_traces(
#     marker=dict(symbol="star", line=dict(width=0.5, color='DarkSlateGrey')),
#     selector=dict(mode="markers"),
    
# )
# bubble_data2_1.layout.xaxis.tickformat = '0%'
# bubble_data2_1.layout.yaxis.tickformat = '0%'
# # Aggiungo y = x
# #x = [0.3,0.4,0.5,0.6,0.7]
# #bubble_data2_1.add_scatter(x=x, y=x, mode="lines", line_color="black", line_width=2, opacity=1, line_dash="dot", name="equilibrium")
# bubble_data2_1.show()

## 7. ~~Bubble plot - 4 categories - summarized~~

Try to summarize data by taking the mean of elements  weight and components weight for each different rank.

**Attention:** the problem is that not all competition and program have the same amount of participiants: consider using another metric rather than rank! (example of votes)

In [ ]:
# medie = data.groupby(['category', 'rank'])['weight_elem', 'weight_comp'].mean().reset_index()
# medie.head()

In [ ]:
# medie.tail()

In [ ]:
# # Ordino data secondo il rank in modo crescente:
# medie = medie.sort_values(by=['rank'], ascending=True)
# # Create a new column with the inverse of the rank:
# medie['rank_inv'] = 1/medie['rank']
# medie.colnames = ['category', 'rank', 'weight_elem', 'weight_comp', 'rank_inv']
# medie.head(1)

In [ ]:
# bubble_medie = px.scatter(
#     medie, y="weight_comp", x="weight_elem",
#     color="category", size="rank_inv", 
#     title='Rank: components vs elements score for each performance',
#     size_max=20,
#     labels={
#         "weight_comp": "Components weight mean",
#         "weight_elem": "Elements weight mean",
#         "rank": "Rank", 
#         "category": "Category"
#     },
#     hover_data={"rank":True, "rank_inv":False},
#     category_orders={ # replaces default order by column name
#         "category": ['Ladies','Men','Pairs','Ice dance']
#     },
#     template="simple_white",
#     width=1000,
#     height=600
#     )
# bubble_medie.update_traces(
#     marker=dict(symbol="star", line=dict(width=0.5, color='DarkSlateGrey')),
#     selector=dict(mode="markers"),
    
# )
# bubble_medie.layout.xaxis.tickformat = '0%'
# bubble_medie.layout.yaxis.tickformat = '0%'
# # Aggiungo y = x
# #x = [0.3,0.4,0.5,0.6,0.7]
# #bubble_medie.add_scatter(x=x, y=x, mode="lines", line_color="black", line_width=2, opacity=1, line_dash="dot", name="equilibrium")
# bubble_medie.show()

## 8. Questioni aperte

* metrica rank cambiare
* plot tipo barra status 

## 9. Bubble plot - summarized + new version

* inverso asse rank
* usare nuova metrica -> alto, medio alto, medio, medio-basso, basso
* provare anche il grafico delle barre
* sommare classifica short + long program

In [39]:
bubble_medie3 = px.scatter(
    medie, x="weight_elem", y="rank",
    color="category", #size="rank_inv", 
    title='Rank: components vs elements score for each performance',
    log_x=False,
    labels={
        "category": "Category",  
        "rank": "Rank", 
        "program": "Program",
        "weight_elem": "Weight of elements"
    },
    #hover_data={ "rank":True, "program":True, "rank_inv":False, "total_segment_score":True},
    category_orders={ # replaces default order by column name
        "category": ['Ladies','Ladies free','Men','Men free','Pairs','Pairs free','Ice dance']
    },
    template="simple_white",
    width=1000,
    height=600
)
bubble_medie3.update_traces(
    marker=dict(symbol="star", line=dict(width=0.5, color='DarkSlateGrey'), size=10),
    selector=dict(mode="markers"),
    
)
bubble_medie3['layout']['yaxis']['autorange'] = "reversed"
# bubble_medie3.update_yaxes(range = [1,37])
# Show minimum and maximum values on axis y:
bubble_medie3.update_yaxes(range = [1,37], tickmode = 'array', tickvals = np.arange(1,38,2))
bubble_medie3.layout.xaxis.tickformat = '0%'
# #Aggiungo y = x
bubble_medie3.add_vline(x=0.5, line_width=2, line_dash="dash", line_color="black", opacity=1, name="equilibrium")
# x = [0,10,20,30,40,50,60,70,80,90,100,110,120]
# bubble_data2.add_scatter(x=x, y=x, mode="lines", line_color="black", line_width=2, opacity=1, line_dash="dot", name="equilibrium")
bubble_medie3.show()

## 10. Bubble plot - summarized + new version + new metric

In [40]:
# Calcola quante categorie presenta la colonna 'rank' in 'data' per ogni 'competition' e 'program':
ranking.head(20) #Series object

competition                                     program                     
Grand Prix Final 2017 Senior and Junior         ICE DANCE FREE DANCE             6
                                                ICE DANCE SHORT DANCE            6
                                                JUNIOR ICE DANCE FREE DANCE      6
                                                JUNIOR ICE DANCE SHORT DANCE     6
                                                JUNIOR LADIES FREE SKATING       6
                                                JUNIOR LADIES SHORT PROGRAM      6
                                                JUNIOR MEN FREE SKATING          6
                                                JUNIOR MEN SHORT PROGRAM         6
                                                JUNIOR PAIRS FREE SKATING        6
                                                JUNIOR PAIRS SHORT PROGRAM       6
                                                LADIES FREE SKATING              6
          

In [42]:
# get info about the ranking:
ranking.describe() # min is 5 max is 37 of number of participants in all competitions of all programs

count    152.000000
mean      11.355263
std        6.493408
min        5.000000
25%        7.000000
50%       10.000000
75%       12.000000
max       37.000000
Name: rank, dtype: float64

In [61]:
# Recall: ranking = data.groupby(['competition','program'])['rank'].nunique()
# Crea una nuova colonna in 'data' con il numero di partecipanti per ogni 'program' di ogni 'competition' usando la serie 'ranking':
data['n_participants'] = data.apply(lambda x: ranking.loc[(x['competition'], x['program'])], axis=1)
data.head(1)

# Crea una colonna con il rank percentuale:
data['rank_perc'] = data['rank']/data['n_participants']

# Crea una colonna con il rank percentuale mappato in una variabile categorica:
data['rank_perc_cat'] = data['rank_perc'].apply(lambda x: 'high' if x <= 0.20 else ('medium-high' if x <= 0.4 else ('medium' if x <= 0.60 else ('medium-low' if x <= 0.8 else 'low'))))

# Creo il dataset medie2 coi dati dei valori medi delle colonne 'weight_elem', 'weight_comp' raggruppati secondo 'category' e 'rank_perc_cat':
medie2 = data.groupby(['category', 'rank_perc_cat'])['weight_elem', 'weight_comp'].mean().reset_index()

# Aggiungo una colonna 'rank_perc' a medie2 matchando con la colonna 'rank_perc' di data:
medie2['rank_perc'] = medie2.apply(lambda x: data.loc[(data['category'] == x['category']) & (data['rank_perc_cat'] == x['rank_perc_cat']), 'rank_perc'].iloc[0], axis=1)
# Ordino medie2 secondo il rank percentuale in modo crescente:
medie2 = medie2.sort_values(by=['rank_perc'], ascending=True)



# Create a new column with the inverse of the rank:
#medie2['rank_inv'] = 1/medie2['rank']
#medie2.colnames = ['category', 'rank', 'weight_elem', 'weight_comp', 'rank_inv', 'rank_perc', 'rank_perc_cat']
medie2.head()

<ipython-input-61-b4687c6eeb91>:13: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,category,rank_perc_cat,weight_elem,weight_comp,rank_perc
5,Ladies,high,0.534455,0.465871,0.029412
0,Ice dance,high,0.508184,0.492109,0.031250
15,Men,high,0.542448,0.459869,0.090909
20,Men free,high,0.528909,0.474816,0.100000
25,Pairs,high,0.548019,0.453458,0.125000


In [62]:
medie2.tail()

,category,rank_perc_cat,weight_elem,weight_comp,rank_perc
11,Ladies free,low,0.475213,0.535660,0.833333
21,Men free,low,0.473782,0.536319,0.833333
31,Pairs free,low,0.496545,0.513525,0.833333
1,Ice dance,low,0.503570,0.502120,0.833333
26,Pairs,low,0.535816,0.476752,0.833333


In [66]:
bubble_medie2_1 = px.scatter(
    medie2, x="weight_elem", y="rank_perc_cat",
    color="category", #size="rank_inv", 
    title='Rank: components vs elements score for each performance',
    log_x=False,
    labels={
        "category": "Category",  
        "rank_perc_cat": "Rank", 
        "weight_elem": "Weight of elements",
        "rank_perc": "Rank percentage"
    },
    hover_data={"rank_perc":True},
    category_orders={ # replaces default order by column name
        "category": ['Ladies','Ladies free','Men','Men free','Pairs','Pairs free','Ice dance']
    },
    template="simple_white",
    width=1000,
    height=600
)
bubble_medie2_1.update_traces(
    marker=dict(symbol="star", line=dict(width=0.5, color='DarkSlateGrey'), size=18),
    selector=dict(mode="markers"),
    
)
bubble_medie2_1['layout']['yaxis']['autorange'] = "reversed"
# bubble_medie3.update_yaxes(range = [1,37])
# Show order of the 5 categories on axis y:
#bubble_medie2_1.update_yaxes(range = [1,5], tickmode = 'array', tickvals = np.arange(1,6,1))
#bubble_medie2_1.update_yaxes(range = [0,4], tickmode = 'array', tickvals = np.arange(0,5,1), ticktext = ['high', 'medium-high', 'medium', 'medium-low', 'low'])
bubble_medie2_1.layout.xaxis.tickformat = '0%'
# #Aggiungo y = x
bubble_medie2_1.add_vline(x=0.5, line_width=2, line_dash="dash", line_color="black", opacity=1, name="equilibrium")
# x = [0,10,20,30,40,50,60,70,80,90,100,110,120]
# bubble_data2.add_scatter(x=x, y=x, mode="lines", line_color="black", line_width=2, opacity=1, line_dash="dot", name="equilibrium")
bubble_medie2_1.show()